In [1]:

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_score, f1_score
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import recall_score
from sklearn import metrics
from tensorflow.python.ops.numpy_ops import np_config
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import seaborn as sns
import pandas as pd
import keras
import io

In [2]:
tf.__version__

'2.17.1'

In [3]:
%load_ext tensorboard

In [4]:
logdir='log'

In [5]:


(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

train_images, test_images = train_images / 255.0, test_images / 255.0

classes=[0,1,2,3,4,5,6,7,8,9]



11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [6]:


model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))




/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:


tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)



In [ ]:


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x=train_images,
            y=train_labels,
            epochs=20,

validation_data
=(test_images, test_labels))



Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 43ms/step - accuracy: 0.8938 - loss: 0.3469 - val_accuracy: 0.9818 - val_loss: 0.0576
Epoch 2/20
1039/1875 ━━━━━━━━━━━━━━━━━━━━ 23s 28ms/step - accuracy: 0.9857 - loss: 0.0470

In [ ]:


# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()



In [ ]:


y_true=test_labels
#y_pred=model.predict_classes(test_images)
y_pred= np.argmax(model.predict(test_images), axis=-1)




y_true=test_labels
#y_pred=model.predict_classes(test_images)
y_pred= np.argmax(model.predict(test_images), axis=-1)




In [ ]:


classes=[0,1,2,3,4,5,6,7,8,9]



In [ ]:


con_mat = tf.math.confusion_matrix(labels=y_true, predictions=y_pred).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                     index = classes,
                     columns = classes)

figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.title("Confusion Matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()



In [ ]:


model1 = models.Sequential()
model1.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Conv2D(64, (3, 3), activation='relu'))

model1.add(layers.Flatten())
model1.add(layers.Dense(64, activation='relu'))
model1.add(layers.Dense(10))

model1.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']

)

model.summary()



In [ ]:


file_writer = tf.summary.create_file_writer(logdir + '/cm')

def log_confusion_matrix(epoch, logs):
  # Use the model to predict the values from the validation dataset.
  test_pred = np.argmax(model1.predict(test_images), axis=-1)

  con_mat = tf.math.confusion_matrix(labels=y_true, predictions=y_pred).numpy()
  con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

  con_mat_df = pd.DataFrame(con_mat_norm,
                     index = classes,
                     columns = classes)

  figure = plt.figure(figsize=(8, 8))
  sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')

  buf = io.BytesIO()
  plt.savefig(buf, format='png')

  plt.close(figure)
  buf.seek(0)
  image = tf.image.decode_png(buf.getvalue(), channels=4)

  image = tf.expand_dims(image, 0)

  # Log the confusion matrix as an image summary.
  with file_writer.as_default():
    tf.summary.image("Confusion Matrix", image, step=epoch)


logdir='logs/images'

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)



In [ ]:


# Acurácia
acuracia = 100*accuracy_score(y_true, y_pred)
print('Acurácia: {:.2f}%' .format(acuracia))



In [ ]:


# F1-Score
f1_macro = 100*f1_score(y_true, y_pred, average='macro')
f1_micro = 100*f1_score(y_true, y_pred, average='micro')
f1_weighted = 100*f1_score(y_true, y_pred, average='weighted')
print('F-Score: {:.4f}%' .format(f1_macro))
print('F-Score: {:.4f}%' .format(f1_micro))
print('F-Score: {:.4f}%' .format(f1_weighted))



In [ ]:


# Precisão

precision_macro = 100*precision_score(y_true, y_pred, average='macro')
precision_micro = 100*precision_score(y_true, y_pred, average='micro')
precision_weighted = 100*precision_score(y_true, y_pred, average='weighted')
print('Precisão Macro:    {:.3f}%' .format(precision_macro ))
print('Precisão Micro:    {:.3f}%' .format(precision_micro))
print('Precisão Weighted: {:.3f}%' .format(precision_weighted))



In [ ]:



Sensitivity_recall_macro = 100*metrics.recall_score(y_true, y_pred, average='macro')
Sensitivity_recall_micro = 100*metrics.recall_score(y_true, y_pred, average='micro')
Sensitivity_recall_weighted = 100*metrics.recall_score(y_true, y_pred, average='weighted')


print('Sensibilidade Macro:    {:.4f}%' .format(Sensitivity_recall_macro))
print('Sensibilidade Micro:    {:.4f}%' .format(Sensitivity_recall_micro))
print('Sensibilidade Weighted: {:.4f}%' .format(Sensitivity_recall_weighted))





In [ ]:



Specificity_macro = 100*metrics.recall_score(y_pred, y_true, average='macro', pos_label=1)
Specificity_micro = 100*metrics.recall_score(y_pred, y_true, average='micro', pos_label=1)
Specificity_weighted = 100*metrics.recall_score(y_pred, y_true, average='weighted', pos_label=1)


print('Especificidade Macro    {:.4f}%' .format(Specificity_macro))
print('Especificidade Micro    {:.4f}%' .format(Specificity_micro))
print('Especificidade Weighted {:.4f}%' .format(Specificity_weighted))




In [ ]:


print(classification_report(y_true, y_pred))



In [ ]:


# Start TensorBoard.
%tensorboard --logdir logs/images

